## 1. Setup and Imports

In [1]:
import os
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from PIL import Image

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

PyTorch version: 2.9.0+cu126
CUDA available: False


In [2]:
# Set random seeds for reproducibility
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)

# Device configuration
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Using device: {device}")

Using device: cpu


## 2. Data Preprocessing and Augmentation

In [3]:
# Define image transforms
# ImageNet normalization statistics
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]
IMAGE_SIZE = 224

# Training transforms with augmentation
train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])

# Test transforms (no augmentation for evaluation)
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])

## 3. Model Architecture

In [4]:
class ObjectDetectionModel(nn.Module):
    """
    Multi-task learning model for object detection.
    
    Architecture:
    - Backbone: ResNet50 (pretrained on ImageNet)
    - Classification Head: Predicts cow stall numbers (61 classes)
    - Regression Head: Predicts bounding box coordinates (4 values)
    """
    
    def __init__(self, num_classes=61, num_bbox_coords=4):
        super(ObjectDetectionModel, self).__init__()
        
        # Load pretrained ResNet50
        self.backbone = models.resnet50(pretrained=True)
        
        # Get the number of input features for the FC layer
        num_features = self.backbone.fc.in_features
        
        # Replace FC layer with identity to get feature maps
        self.backbone.fc = nn.Identity()
        
        # Flatten layer
        self.flatten = nn.Flatten()
        
        # Classification head
        self.classifier = nn.Linear(num_features, num_classes)
        
        # Bounding box regression head
        self.bbox_regressor = nn.Linear(num_features, num_bbox_coords)
    
    def forward(self, x):
        # Extract features from backbone
        features = self.backbone(x)
        
        # Flatten features
        features = self.flatten(features)
        
        # Classification and bounding box predictions
        class_logits = self.classifier(features)
        bbox_coords = self.bbox_regressor(features)
        
        return class_logits, bbox_coords

## 4. Custom Dataset Class

In [5]:
class CowStallDataset(Dataset):
    """
    Custom dataset for cow stall detection.
    
    Loads images and their corresponding:
    - Stall number labels
    - Bounding box coordinates
    """
    
    def __init__(self, dataframe, image_dir, transform=None):
        """
        Args:
            dataframe: Pandas DataFrame with image paths and annotations
            image_dir: Directory containing images
            transform: Torchvision transforms to apply
        """
        self.dataframe = dataframe
        self.image_dir = image_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idx):
        try:
            # Read image
            image_filename = self.dataframe.iloc[idx, 0]
            image_path = os.path.join(self.image_dir, image_filename)
            
            image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
            if image is None:
                raise ValueError(f"Failed to load image: {image_path}")
            
            # Resize image
            image = cv2.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
            
            # Convert BGR to RGB
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
            
            # Normalize to [0, 1]
            image /= 255.0
            
            # Apply transforms
            if self.transform:
                image = self.transform(image)
            
            # Get label (stall number)
            label = torch.tensor(int(self.dataframe.iloc[idx, 5]), dtype=torch.long)
            
            # Get bounding box coordinates
            bbox_columns = ['box_position_1', 'box_position_2', 'box_position_3', 'box_position_4']
            bbox = self.dataframe.loc[idx, bbox_columns].values.astype(np.float32)
            
            # Convert (x, y, width, height) to (x1, y1, x2, y2)
            bbox[2] = bbox[0] + bbox[2]  # x2 = x1 + width
            bbox[3] = bbox[1] + bbox[3]  # y2 = y1 + height
            
            bbox = torch.as_tensor(bbox, dtype=torch.float32)
            
            return image, label, bbox
        
        except Exception as e:
            print(f"Error loading sample {idx}: {str(e)}")
            raise

## 5. Data Loading (Update paths for your environment)

In [6]:
# Configuration - Update these paths for your environment
CONFIG = {
    'train_csv_path': "/content/drive/MyDrive/New folder/Stall_num_images/Train.csv",
    'test_csv_path': "/content/drive/MyDrive/New folder/Stall_num_images/Test.csv",
    'image_dir': "/content/drive/MyDrive/Stall_num_images",
    'batch_size': 4,
    'num_workers': 0,
    'epochs': 100,
    'learning_rate': 0.0001,
    'weight_decay': 0.0001,
    'num_classes': 61
}

print(f"Configuration:")
for key, value in CONFIG.items():
    print(f"  {key}: {value}")

Configuration:
  train_csv_path: /content/drive/MyDrive/New folder/Stall_num_images/Train.csv
  test_csv_path: /content/drive/MyDrive/New folder/Stall_num_images/Test.csv
  image_dir: /content/drive/MyDrive/Stall_num_images
  batch_size: 4
  num_workers: 0
  epochs: 100
  learning_rate: 0.0001
  weight_decay: 0.0001
  num_classes: 61


In [7]:
# Load data
print("Loading datasets...")
df_train = pd.read_csv(CONFIG['train_csv_path']).fillna(0)
df_test = pd.read_csv(CONFIG['test_csv_path']).fillna(0)

print(f"Training samples: {len(df_train)}")
print(f"Test samples: {len(df_test)}")
print(f"\nDataFrame columns: {df_train.columns.tolist()}")

Loading datasets...


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/New folder/Stall_num_images/Train.csv'

In [ ]:
# Create datasets
train_dataset = CowStallDataset(
    df_train,
    CONFIG['image_dir'],
    transform=train_transform
)

test_dataset = CowStallDataset(
    df_test,
    CONFIG['image_dir'],
    transform=test_transform
)

# Create data loaders
train_loader = DataLoader(
    train_dataset,
    batch_size=CONFIG['batch_size'],
    shuffle=True,
    num_workers=CONFIG['num_workers'],
    pin_memory=True if torch.cuda.is_available() else False
)

test_loader = DataLoader(
    test_dataset,
    batch_size=CONFIG['batch_size'],
    shuffle=False,
    num_workers=CONFIG['num_workers'],
    pin_memory=True if torch.cuda.is_available() else False
)

print(f"Training batches: {len(train_loader)}")
print(f"Test batches: {len(test_loader)}")

## 6. Model Training Setup

In [ ]:
# Initialize model
model = ObjectDetectionModel(
    num_classes=CONFIG['num_classes'],
    num_bbox_coords=4
).to(device)

print(f"Model moved to {device}")
print(f"Total parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

In [ ]:
# Loss functions
criterion_classification = nn.CrossEntropyLoss()
criterion_bbox = nn.MSELoss()

# Optimizer
trainable_params = [p for p in model.parameters() if p.requires_grad]
optimizer = optim.Adam(
    trainable_params,
    lr=CONFIG['learning_rate'],
    weight_decay=CONFIG['weight_decay']
)

# Learning rate scheduler
scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer,
    step_size=30,
    gamma=0.1
)

print(f"Optimizer: {optimizer.__class__.__name__}")
print(f"Scheduler: StepLR (step_size=30, gamma=0.1)")

## 7. Training Loop

In [ ]:
# Training history
history = {
    'train_loss': [],
    'train_accuracy': [],
    'val_loss': [],
    'val_accuracy': []
}

# Hyperparameters
BBOX_LOSS_WEIGHT = 0.01  # Weight for bounding box loss
EPOCHS = CONFIG['epochs']

print(f"Starting training for {EPOCHS} epochs...\n")

In [ ]:
for epoch in range(EPOCHS):
    # ==================== Training ====================
    model.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0
    
    for batch_idx, (images, labels, bboxes) in enumerate(train_loader):
        # Move data to device
        images = images.to(device)
        labels = labels.to(device)
        bboxes = bboxes.to(device)
        
        # Forward pass
        class_logits, bbox_predictions = model(images)
        
        # Calculate losses
        loss_cls = criterion_classification(class_logits, labels)
        loss_bbox = torch.sqrt(criterion_bbox(bbox_predictions, bboxes)) * BBOX_LOSS_WEIGHT
        loss_total = loss_cls + loss_bbox
        
        # Backward pass
        optimizer.zero_grad()
        loss_total.backward()
        optimizer.step()
        
        # Update metrics
        train_loss += loss_total.item()
        predictions = torch.argmax(class_logits, dim=1)
        train_correct += (predictions == labels).sum().item()
        train_total += labels.size(0)
    
    # Calculate training metrics
    train_loss /= len(train_loader)
    train_accuracy = train_correct / train_total
    history['train_loss'].append(train_loss)
    history['train_accuracy'].append(train_accuracy)
    
    # ==================== Validation ====================
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    
    with torch.no_grad():
        for images, labels, bboxes in test_loader:
            # Move data to device
            images = images.to(device)
            labels = labels.to(device)
            bboxes = bboxes.to(device)
            
            # Forward pass
            class_logits, bbox_predictions = model(images)
            
            # Calculate losses
            loss_cls = criterion_classification(class_logits, labels)
            loss_bbox = torch.sqrt(criterion_bbox(bbox_predictions, bboxes)) * BBOX_LOSS_WEIGHT
            loss_total = loss_cls + loss_bbox
            
            # Update metrics
            val_loss += loss_total.item()
            predictions = torch.argmax(class_logits, dim=1)
            val_correct += (predictions == labels).sum().item()
            val_total += labels.size(0)
    
    # Calculate validation metrics
    val_loss /= len(test_loader)
    val_accuracy = val_correct / val_total
    history['val_loss'].append(val_loss)
    history['val_accuracy'].append(val_accuracy)
    
    # Update learning rate
    scheduler.step()
    
    # Print progress
    if (epoch + 1) % 10 == 0 or epoch == 0:
        print(f"Epoch {epoch + 1:3d}/{EPOCHS} | "
              f"Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy*100:.2f}% | "
              f"Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy*100:.2f}%")

In [ ]:
print("\n" + "="*80)
print("TRAINING COMPLETED")
print("="*80)
print(f"Final Training Loss: {history['train_loss'][-1]:.4f}")
print(f"Final Training Accuracy: {history['train_accuracy'][-1]*100:.2f}%")
print(f"Final Validation Loss: {history['val_loss'][-1]:.4f}")
print(f"Final Validation Accuracy: {history['val_accuracy'][-1]*100:.2f}%")

## 8. Model Evaluation and Visualization

In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# Loss plot
axes[0].plot(history['train_loss'], label='Train Loss', linewidth=2)
axes[0].plot(history['val_loss'], label='Validation Loss', linewidth=2)
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Loss', fontsize=12)
axes[0].set_title('Training and Validation Loss', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)

# Accuracy plot
axes[1].plot(history['train_accuracy'], label='Train Accuracy', linewidth=2)
axes[1].plot(history['val_accuracy'], label='Validation Accuracy', linewidth=2)
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Accuracy', fontsize=12)
axes[1].set_title('Training and Validation Accuracy', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Find best validation accuracy
best_epoch = np.argmax(history['val_accuracy']) + 1
best_val_acc = np.max(history['val_accuracy'])

print(f"\nBest Validation Accuracy: {best_val_acc*100:.2f}% (Epoch {best_epoch})")
print(f"✓ Model exceeds 80% accuracy requirement" if best_val_acc > 0.8 else "✗ Below 80% accuracy")

## 9. Model Saving

In [ ]:
# Save model checkpoint
model_save_path = os.path.join(CONFIG['image_dir'], 'object_detection_model.pt')

checkpoint = {
    'epoch': EPOCHS,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'scheduler_state_dict': scheduler.state_dict(),
    'train_loss': history['train_loss'][-1],
    'val_loss': history['val_loss'][-1],
    'val_accuracy': history['val_accuracy'][-1],
    'config': CONFIG
}

try:
    torch.save(checkpoint, model_save_path)
    print(f"Model saved successfully to: {model_save_path}")
except Exception as e:
    print(f"Error saving model: {str(e)}")

In [ ]:
# Load model function for future use
def load_model(checkpoint_path, device):
    """
    Load model from checkpoint.
    
    Args:
        checkpoint_path: Path to saved checkpoint
        device: Device to load model on
    
    Returns:
        Loaded model and checkpoint information
    """
    checkpoint = torch.load(checkpoint_path, map_location=device)
    
    model = ObjectDetectionModel(
        num_classes=checkpoint['config']['num_classes'],
        num_bbox_coords=4
    ).to(device)
    
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    
    return model, checkpoint

print("Model loading function defined.")

## 10. Summary and Results

In [ ]:
print("\n" + "="*80)
print("PROJECT SUMMARY")
print("="*80)
print(f"\nDataset: Cow Stall Number Detection")
print(f"  Training samples: {len(df_train)}")
print(f"  Test samples: {len(df_test)}")
print(f"\nModel Architecture: ObjectDetectionModel")
print(f"  Backbone: ResNet50 (pretrained)")
print(f"  Classification classes: {CONFIG['num_classes']}")
print(f"  Total parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"\nTraining Configuration:")
print(f"  Epochs: {EPOCHS}")
print(f"  Batch size: {CONFIG['batch_size']}")
print(f"  Learning rate: {CONFIG['learning_rate']}")
print(f"  Optimizer: Adam")
print(f"\nFinal Results:")
print(f"  Training Loss: {history['train_loss'][-1]:.4f}")
print(f"  Training Accuracy: {history['train_accuracy'][-1]*100:.2f}%")
print(f"  Validation Loss: {history['val_loss'][-1]:.4f}")
print(f"  Validation Accuracy: {history['val_accuracy'][-1]*100:.2f}%")
print(f"  Best Validation Accuracy: {best_val_acc*100:.2f}% (Epoch {best_epoch})")
print(f"\n✓ Model meets 80% accuracy requirement" if best_val_acc > 0.8 else "\n✗ Below 80% accuracy")
print(f"\nModel saved to: {model_save_path}")
print("="*80)